In [109]:
# Base libraries
import os
import time
import datetime
import json

# Scientific libraries
import numpy as np
import pandas as pd
from empiricaldist import Cdf, Pmf

# from sklearn import svm # or any other function

# Visual libraries
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import missingno as msno  # Visualize missing values
# most definitely plotly

# Helper libraries
from tqdm.notebook import tqdm, trange
from colorama import Fore, Back, Style
import warnings
warnings.filterwarnings('ignore')

# Visual setup
plt.style.use('fivethirtyeight')   # alternatives below
rcParams['axes.spines.right'] = False
rcParams['axes.spines.top'] = False
plt.rc('xtick', labelsize=11)
plt.rc('ytick', labelsize=11)
custom_colors = ['#74a09e','#86c1b2','#98e2c6','#f3c969','#f2a553', '#d96548', '#c14953']
sns.set_palette(custom_colors)
%config InlineBackend.figure_format = 'retina'
%config Completer.use_jedi = False

# Pandas options
pd.set_option('max_colwidth', 40)
pd.options.display.max_columns = None  # Possible to limit
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

df_flights = pd.read_csv('flights.csv')

In [110]:
df_flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
df_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

### Data Dictionary

In [112]:
import os

class create_data_dictionary:
    def __init__(self):
        '''This class provides functions to quickly develop a data dictionary for a data set'''
        pass

    def make_my_data_dictionary(self, dataFrame):
        '''Create an initial data dictionary excluding definitions for the meaning of features'''
        col_ = dataFrame.columns
        df_DataDict = {}

        for col in col_:
            total_count = len(dataFrame[col])
            null_count = sum(dataFrame[col].isna())
            null_percentage = (null_count / total_count) * 100

            df_DataDict[col] = {
                'Column Name': col,
                'Definition': str(''),
                'Type': str(dataFrame.dtypes[col]),
                'Length': total_count,
                'Null_Count': null_count,
                'Null_Percentage': null_percentage,
                'Size(Memory)': dataFrame.memory_usage()[col]
            }

        df_DD = pd.DataFrame(df_DataDict).transpose()

        return df_DD

    def define_data_meaning(self, df_data_dictionary):
        '''Quickly provide input regarding each column's meaning and transpose into a usable dictionary'''
        col_ = df_data_dictionary.columns
        d = 'Definition'

        for col in col_:
            if col != 'Column Name' and col != 'Definition':
                df_data_dictionary[col][d] = input('Provide a data definition for {}'.format(col))

        return df_data_dictionary

    def update_dd_definition(self, df_data_dictionary, attribute):
        try:
            df_dd = df_data_dictionary.transpose()
            df_dd[attribute]['Definition'] = input('Provide a data definition for {}'.format(attribute))
            return df_dd.transpose()
        except:
            print('Sorry, there was an error. Check the attribute name and try again')

    def export_to_csv(self, df_data_dictionary, original_file_path):
        '''Export the data dictionary to a CSV file'''
        file_name = os.path.basename(original_file_path)
        csv_filename = 'Data_Dictionary_{}.csv'.format(os.path.splitext(file_name)[0])
        df_data_dictionary.to_csv(csv_filename, index=False)
        print('Data dictionary exported to {}'.format(csv_filename))

Created a data dictionary

In [113]:
# dd = create_data_dictionary()
# df_flights_dd = dd.make_my_data_dictionary(df_flights)
# df_flights_dd = dd.define_data_meaning(df_flights_dd)

# print(df_flights_dd)

# # Export to CSV
# original_file_path = r'C:\Users\nickb\Documents\Data Science\Projects\2024\2015FlightDelaysAndCancellations'
# dd.export_to_csv(df_flights_dd, original_file_path)

In [114]:
# # Create a loop so that any column that contains DELAY has the value counts shown and a distribution count is shown
# for col in df_flights.columns:
#     if 'DELAY' in col:
#         print(df_flights[col].value_counts())
#         print('\n')

# # Create a loop so that any column that contains DELAY has the value counts shown and a distribution count is shown
# for col in df_flights.columns:
#     if 'DELAY' in col:
#         print(df_flights[col].value_counts(normalize=True))
#         print('\n')


# # Plot a histogram of the distribution of the arrival delay times
# df_flights['ARRIVAL_DELAY'].hist(bins=100)
    

### A. Cleaning Dates & Times

In [115]:
# Converting Columns into Single Date Column
df_flights['DATE'] = pd.to_datetime(df_flights[['YEAR','MONTH', 'DAY']])

In [116]:

def convert_time_string_to_time(time_str):
    """
    Convert a time string 'HHMM' to a datetime.time object.
    """
    if pd.isnull(time_str):
        return np.nan

    if time_str == 2400:
        time_str = 0

    time_str = "{0:04d}".format(int(time_str))
    return datetime.time(int(time_str[0:2]), int(time_str[2:4]))

def combine_date_and_time(date, time):
    """
    Combine a date and time to produce a datetime.datetime object.
    """
    if pd.isnull(date) or pd.isnull(time):
        return np.nan

    return datetime.datetime.combine(date, time)

def create_flight_datetime(df, time_col):
    """
    Combine date and time columns of a dataframe to create datetime objects.
    """
    def process_row(row):
        date, time = row['DATE'], row[time_col]
        if pd.isnull(time):
            return np.nan
        if float(time) == 2400:
            date += datetime.timedelta(days=1)
            time = datetime.time(0, 0)
        else:
            time = convert_time_string_to_time(time)
        return combine_date_and_time(date, time)

    return df[['DATE', time_col]].apply(process_row, axis=1)

In [117]:
# Update the 'SCHEDULED_DEPARTURE' column
df_flights['SCHEDULED_DEPARTURE'] = create_flight_datetime(df_flights, 'SCHEDULED_DEPARTURE')

# Update 'DEPARTURE_TIME', 'SCHEDULED_ARRIVAL', and 'ARRIVAL_TIME' columns
df_flights['DEPARTURE_TIME'] = df_flights['DEPARTURE_TIME'].apply(convert_time_string_to_time)
df_flights['SCHEDULED_ARRIVAL'] = df_flights['SCHEDULED_ARRIVAL'].apply(convert_time_string_to_time)
df_flights['ARRIVAL_TIME'] = df_flights['ARRIVAL_TIME'].apply(convert_time_string_to_time)

In [118]:
df_flights.iloc[:, 9:22].head()

,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME
0,2015-01-01 00:05:00,23:54:00,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,04:30:00,04:08:00
1,2015-01-01 00:10:00,00:02:00,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,07:50:00,07:41:00
2,2015-01-01 00:20:00,00:18:00,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,08:06:00,08:11:00
3,2015-01-01 00:20:00,00:15:00,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,08:05:00,07:56:00
4,2015-01-01 00:25:00,00:24:00,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,03:20:00,02:59:00


We can see that Arrival Time Values don't contain any decimal values. As such, we'll convert this column to int for the sake of memory efficiency.

In [119]:
df_flights[df_flights['ARRIVAL_DELAY'] % 1 != 0].head()

# See if there are any ARRIVAL_DELAY values that contain a number with a decimal
df_flights_NAN_or_non_int = df_flights[df_flights['ARRIVAL_DELAY'] % 1 != 0]

# See if there are any ARRIVAL_DELAY values that are NOT NAN
df_flights_NAN_or_non_int[df_flights_NAN_or_non_int['ARRIVAL_DELAY'].notna()]

# Convert Arrival_Delay column to Int. Int64 lets you convert column that contains NaN
# df_flights['ARRIVAL_DELAY'] = df_flights['ARRIVAL_DELAY'].astype('nt64')

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
32,2015,1,1,4,AS,136,N431AS,ANC,SEA,2015-01-01 01:35:00,NaN,NaN,NaN,NaN,205.0,NaN,NaN,1448,NaN,NaN,06:00:00,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN,2015-01-01
42,2015,1,1,4,AA,2459,N3BDAA,PHX,DFW,2015-01-01 02:00:00,NaN,NaN,NaN,NaN,120.0,NaN,NaN,868,NaN,NaN,05:00:00,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN,2015-01-01
68,2015,1,1,4,OO,5254,N746SK,MAF,IAH,2015-01-01 05:10:00,NaN,NaN,NaN,NaN,87.0,NaN,NaN,429,NaN,NaN,06:37:00,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN,2015-01-01
82,2015,1,1,4,MQ,2859,N660MQ,SGF,DFW,2015-01-01 05:25:00,NaN,NaN,NaN,NaN,95.0,NaN,NaN,364,NaN,NaN,07:00:00,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN,2015-01-01
90,2015,1,1,4,OO,5460,N583SW,RDD,SFO,2015-01-01 05:30:00,NaN,NaN,NaN,NaN,90.0,NaN,NaN,199,NaN,NaN,07:00:00,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN,2015-01-01


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE


In [120]:
# SHow Only Rows Where TAIL_Number is Missing
df_flights[df_flights['TAIL_NUMBER'].isna()]

# Fill all rows with a missing Tail # and write 'N/A' (Not Available)
df_flights['TAIL_NUMBER'].fillna('N/A', inplace=True)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
297,2015,1,1,4,F9,865,NaN,MSO,DEN,2015-01-01 06:00:00,NaN,NaN,NaN,NaN,120.0,NaN,NaN,679,NaN,NaN,08:00:00,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN,2015-01-01
298,2015,1,1,4,F9,1256,NaN,ORD,MIA,2015-01-01 06:00:00,NaN,NaN,NaN,NaN,180.0,NaN,NaN,1197,NaN,NaN,10:00:00,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN,2015-01-01
2216,2015,1,1,4,UA,641,NaN,LAX,HNL,2015-01-01 08:45:00,NaN,NaN,NaN,NaN,344.0,NaN,NaN,2556,NaN,NaN,12:29:00,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN,2015-01-01
3490,2015,1,1,4,UA,1412,NaN,ORD,PHL,2015-01-01 10:10:00,NaN,NaN,NaN,NaN,121.0,NaN,NaN,678,NaN,NaN,13:11:00,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN,2015-01-01
3763,2015,1,1,4,US,1883,NaN,SLC,PHL,2015-01-01 10:30:00,NaN,NaN,NaN,NaN,243.0,NaN,NaN,1927,NaN,NaN,16:33:00,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN,2015-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5811608,2015,12,31,4,UA,598,NaN,SMF,DEN,2015-12-31 11:34:00,NaN,NaN,NaN,NaN,148.0,NaN,NaN,909,NaN,NaN,15:02:00,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN,2015-12-31
5813418,2015,12,31,4,UA,338,NaN,PDX,SFO,2015-12-31 13:39:00,NaN,NaN,NaN,NaN,110.0,NaN,NaN,550,NaN,NaN,15:29:00,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN,2015-12-31
5813861,2015,12,31,4,UA,1828,NaN,SEA,IAH,2015-12-31 14:05:00,NaN,NaN,NaN,NaN,258.0,NaN,NaN,1874,NaN,NaN,20:23:00,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN,2015-12-31
5818090,2015,12,31,4,UA,1789,NaN,IAH,TPA,2015-12-31 19:55:00,NaN,NaN,NaN,NaN,125.0,NaN,NaN,787,NaN,NaN,23:00:00,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN,2015-12-31


In [123]:
df_flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,2015-01-01 00:05:00,23:54:00,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,04:30:00,04:08:00,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,2015-01-01 00:10:00,00:02:00,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,07:50:00,07:41:00,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
2,2015,1,1,4,US,840,N171US,SFO,CLT,2015-01-01 00:20:00,00:18:00,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,08:06:00,08:11:00,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,2015-01-01 00:20:00,00:15:00,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,08:05:00,07:56:00,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,2015-01-01 00:25:00,00:24:00,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,03:20:00,02:59:00,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01


In [124]:
df_flights['DAY_OF_WEEK'].value_counts()

4    872521
1    865543
5    862209
3    855897
2    844600
7    817764
6    700545
Name: DAY_OF_WEEK, dtype: int64

In [126]:
# Do Value Counts for Airline
df_flights['AIRLINE'].value_counts()

WN    1261855
DL     875881
AA     725984
OO     588353
EV     571977
UA     515723
MQ     294632
B6     267048
US     198715
AS     172521
NK     117379
F9      90836
HA      76272
VX      61903
Name: AIRLINE, dtype: int64

In [127]:
# Rename AIRLINE to Airline_Abbreviation
df_flights.rename(columns={'AIRLINE':'AIRLINE_ABBREVIATION'}, inplace=True)

Using Tail Number, we'll see what planes flew each flight.
Until XXXX, Tail Numbers needed to be registered every three years. To ensure that the tail numbers being applied to aircraft are accurate, matches were done on tail number + airline. As such, when merging there needs to be two merge conditions met which are 

In [121]:
# # # Import CSV for Aircraft_TGail_Numbers_and_Models
# # df_aircraft = pd.read_csv('Aircraft_Tail_Numbers_and_Models_at_SFO.csv')

# df_aircraft.info()

In [122]:
# #Extract year from Creation_Date and amke column for Creation_Date_Year
# df_aircraft['CREATION_DATE_YEAR'] = pd.DatetimeIndex(df_aircraft['Creation Date']).year

# df_aircraft['CREATION_DATE_YEAR'].value_counts()

# # Do a Left Join on the Aircraft Tail Numbers and Models
# df_flights = df_flights.merge(df_aircraft, how='left', left_on='TAIL_NUMBER', right_on='Tail Number')

# # Show Arrival Delay = 0 and Departure Delay = 0
# df_flights[(df_flights['ARRIVAL_DELAY'] == 0) & (df_flights['DEPARTURE_DELAY'] == 0)].tail(30)

# df_flights['FLIGHT_NUMBER'].value_counts()

# # In Pandas, if you want to only show columns at the beginning and the end you need to do combo slicing
# first_six_columns = df_flights.iloc[:, :8]
# last_six_columns = df_flights.iloc[:, -6:]

# # Concatenating the first and last six columns
# combined_df_flights = pd.concat([first_six_columns, last_six_columns], axis=1)

# # Display the combined DataFrame
# combined_df_flights.head(50)

# (combined_df_flights['Aircraft Model'].isna().sum() / len(combined_df_flights)) * 100

# # Export df_flights to CSV
# # df_flights.to_csv('processed_csvs/flights_cleaned.csv', index=False)
